In [1]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [3]:
import os
proxy='http://127.0.0.1:10809'
os.environ['https_proxy'] = proxy
os.environ['http_proxy'] = proxy
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Processing...
Done!


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [7]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [8]:
data = train_test_split_edges(data)
data

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

In [9]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

In [10]:
from torch_geometric.nn import GAE

In [11]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [12]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [13]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.6193, AP: 0.6609
Epoch: 002, AUC: 0.6393, AP: 0.6753
Epoch: 003, AUC: 0.6463, AP: 0.6828
Epoch: 004, AUC: 0.6513, AP: 0.6885
Epoch: 005, AUC: 0.6513, AP: 0.6895
Epoch: 006, AUC: 0.6520, AP: 0.6912
Epoch: 007, AUC: 0.6533, AP: 0.6934
Epoch: 008, AUC: 0.6540, AP: 0.6958
Epoch: 009, AUC: 0.6546, AP: 0.6980
Epoch: 010, AUC: 0.6550, AP: 0.7004
Epoch: 011, AUC: 0.6553, AP: 0.7027
Epoch: 012, AUC: 0.6550, AP: 0.7047
Epoch: 013, AUC: 0.6548, AP: 0.7066
Epoch: 014, AUC: 0.6543, AP: 0.7088
Epoch: 015, AUC: 0.6537, AP: 0.7107
Epoch: 016, AUC: 0.6534, AP: 0.7116
Epoch: 017, AUC: 0.6527, AP: 0.7124
Epoch: 018, AUC: 0.6523, AP: 0.7128
Epoch: 019, AUC: 0.6525, AP: 0.7139
Epoch: 020, AUC: 0.6525, AP: 0.7142
Epoch: 021, AUC: 0.6527, AP: 0.7148
Epoch: 022, AUC: 0.6530, AP: 0.7151
Epoch: 023, AUC: 0.6533, AP: 0.7155
Epoch: 024, AUC: 0.6543, AP: 0.7165
Epoch: 025, AUC: 0.6556, AP: 0.7176
Epoch: 026, AUC: 0.6580, AP: 0.7190
Epoch: 027, AUC: 0.6614, AP: 0.7207
Epoch: 028, AUC: 0.6663, AP:

In [14]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[-0.2533,  0.2618],
        [ 1.1082, -1.1697],
        [-0.7174,  0.7632],
        ...,
        [ 0.3938, -0.4146],
        [-0.6498,  0.6862],
        [-0.6808,  0.7216]], device='cuda:0', grad_fn=<AddBackward0>)

In [20]:
from torch.utils.tensorboard import SummaryWriter

In [21]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [22]:
writer = SummaryWriter('runs/GAE1_experiment_'+'2d_100_epochs')

In [23]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))


    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6261, AP: 0.6633
Epoch: 002, AUC: 0.6355, AP: 0.6783
Epoch: 003, AUC: 0.6361, AP: 0.6785
Epoch: 004, AUC: 0.6400, AP: 0.6814
Epoch: 005, AUC: 0.6427, AP: 0.6844
Epoch: 006, AUC: 0.6457, AP: 0.6878
Epoch: 007, AUC: 0.6487, AP: 0.6918
Epoch: 008, AUC: 0.6505, AP: 0.6939
Epoch: 009, AUC: 0.6508, AP: 0.6947
Epoch: 010, AUC: 0.6512, AP: 0.6957
Epoch: 011, AUC: 0.6514, AP: 0.6965
Epoch: 012, AUC: 0.6521, AP: 0.6982
Epoch: 013, AUC: 0.6531, AP: 0.7003
Epoch: 014, AUC: 0.6537, AP: 0.7024
Epoch: 015, AUC: 0.6543, AP: 0.7045
Epoch: 016, AUC: 0.6541, AP: 0.7063
Epoch: 017, AUC: 0.6551, AP: 0.7089
Epoch: 018, AUC: 0.6551, AP: 0.7104
Epoch: 019, AUC: 0.6540, AP: 0.7112
Epoch: 020, AUC: 0.6539, AP: 0.7124
Epoch: 021, AUC: 0.6537, AP: 0.7134
Epoch: 022, AUC: 0.6534, AP: 0.7138
Epoch: 023, AUC: 0.6532, AP: 0.7143
Epoch: 024, AUC: 0.6529, AP: 0.7148
Epoch: 025, AUC: 0.6528, AP: 0.7151
Epoch: 026, AUC: 0.6531, AP: 0.7155
Epoch: 027, AUC: 0.6537, AP: 0.7162
Epoch: 028, AUC: 0.6545, AP:

In [24]:
from torch_geometric.nn import VGAE

In [25]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

/home/seeyou/anaconda3/envs/pyg/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [26]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [27]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)

    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [28]:
writer = SummaryWriter('runs/VGAE_experiment_'+'2d_100_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))


    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6022, AP: 0.6294
Epoch: 002, AUC: 0.6319, AP: 0.6565
Epoch: 003, AUC: 0.6403, AP: 0.6643
Epoch: 004, AUC: 0.6436, AP: 0.6676
Epoch: 005, AUC: 0.6452, AP: 0.6693
Epoch: 006, AUC: 0.6456, AP: 0.6700
Epoch: 007, AUC: 0.6457, AP: 0.6701
Epoch: 008, AUC: 0.6452, AP: 0.6699
Epoch: 009, AUC: 0.6448, AP: 0.6698
Epoch: 010, AUC: 0.6446, AP: 0.6699
Epoch: 011, AUC: 0.6443, AP: 0.6697
Epoch: 012, AUC: 0.6438, AP: 0.6698
Epoch: 013, AUC: 0.6430, AP: 0.6692
Epoch: 014, AUC: 0.6422, AP: 0.6685
Epoch: 015, AUC: 0.6412, AP: 0.6678
Epoch: 016, AUC: 0.6404, AP: 0.6676
Epoch: 017, AUC: 0.6397, AP: 0.6670
Epoch: 018, AUC: 0.6387, AP: 0.6664
Epoch: 019, AUC: 0.6378, AP: 0.6660
Epoch: 020, AUC: 0.6374, AP: 0.6658
Epoch: 021, AUC: 0.6376, AP: 0.6661
Epoch: 022, AUC: 0.6375, AP: 0.6662
Epoch: 023, AUC: 0.6380, AP: 0.6669
Epoch: 024, AUC: 0.6389, AP: 0.6675
Epoch: 025, AUC: 0.6400, AP: 0.6683
Epoch: 026, AUC: 0.6411, AP: 0.6695
Epoch: 027, AUC: 0.6424, AP: 0.6701
Epoch: 028, AUC: 0.6435, AP: